# Content
This notebook contains a script to derive soil properties according to:
- Gazetas, G. (1991). Formulas and charts for impedances of surface and embedded foundations. Journal of Geotechnical Engineering, 117(9), 1363–1381. https://doi.org/10.1061/(asce)0733-9410(1991)117:9(1363`
  
Input parameters are from `CIEM5220_SRE Project_Part 2_2025.pdf`

In [6]:
import numpy as np

In [14]:
# Soil Properties (indipendent of Student Number)
nu = 0.33      #[-] Poisson Ratio
c_u = 50e3  #[Pa] Undrained Shear Strength
v_s = 160      #[m/s] Velocity of shear waves in soil
rho = 1800     #[kg/m^3] Soil Density
beta = 0.05    #[-] Soil hysteretic (material) damping (arbitrarily chosen)

# Foundation dimensions (rectangular slab shallow foundation)
side_x = side_y = 4.94 #[m] As a first implementation the slab is as large as the base of the tower
# depth  not necessary

# Structure
omega = 23.047 #[rad/s] Frequency of interest (Mode 2 - Main mode in X direction)

In [41]:
L = side_x/2                                  #[m] Half-length between center of the circumscribed rectangle and side X-direction (major direction)
B = side_y/2                                  #[m] Half-width between center of the circumscribed rectangle and side Y-direction (minor direction)
A_b = side_x*side_y                           #[m] Foundation surface area
I_bx = side_x*side_y**3/12                    #[m^4] Area moment of inertia about x-axis
I_by = side_x**3*side_y/12                    #[m^4] Area moment of inertia about y-axis
I_bz = side_x*side_y*(side_x**2+side_y**2)/12 #[m^4] Area moment of inertia about z-axis
G = c_u                                       #[Pa] Shear modulus
v_la = 3.4*v_s/(np.pi*(1-nu))                 #[m/s] Lymer's analog wave velocity (formula from cited paper)
chi = A_b/(4*L**2)                            #[-] Parameter from cited paper'"Part B - Notebook (Answers 3-6).ipynb"

print("\nINPUT RECAP:")
print(f"L: {L:.3e} m")
print(f"B: {B:.3e} m")
print(f"A_b: {A_b:.3e} m")
print(f"I_bx: {I_bx:.3e} m^4")
print(f"I_by: {I_by:.3e} m^4")
print(f"I_bz: {I_bz:.3e} m^4")
print(f"G: {G:.3e} Pa")
print(f"v_la: {v_la:.3e} m/s")
print(f"chi: {chi:.3e} -")

#Static Stiffness
K_z = (2*G*L/(1-nu))*(0.73+1.54*chi**0.75)
K_y = (2*G*L/(2-nu))*(2+2.50*chi**0.85)
K_x = K_y - (0.2/(0.75-nu))*G*L*(1-(B/L))
K_rx = (G/(1-nu))*I_bx**0.75*(L/B)**0.25*(2.4+0.5*(B/L))
K_ry = (3*G/(1-nu))*I_by**0.75*(L/B)**0.15
K_t = 3.5*G*I_bz**0.75*(B/L)**0.4*(I_bz/B**4)**0.2

# Dynamic Stiffness
a_0 = omega * B / v_s
print(f"a_0: {a_0:.3e} ")

## Table values
k_z_tab = 0.98         #(table Fig.2(a))
k_y_tab = 0.98         #(table Fig.2(b))
k_x_tab = 1            #(constant)
k_rx_tab = 1-0.20*a_0  #(Table 1)
k_ry_tab = 1-0.26*a_0  #(Table 1)
k_t_tab = 1-0.14*a_0   #(Table 1)

## Coefficients
K_z_dyn = K_z*k_z_tab
K_y_dyn = K_y*k_y_tab
K_x_dyn = K_x*k_x_tab
K_rx_dyn = K_rx*k_rx_tab
K_ry_dyn = K_ry*k_ry_tab
K_t_dyn = K_t*k_t_tab

print("\nDynamic Stiffness Coefficients:")
print(f"K_z_dyn: {K_z_dyn:.3e} N/m")
print(f"K_y_dyn: {K_y_dyn:.3e} N/m")
print(f"K_x_dyn: {K_x_dyn:.3e} N/m")
print(f"K_rx_dyn: {K_rx_dyn:.3e} N/m")
print(f"K_ry_dyn: {K_ry_dyn:.3e} N/m")
print(f"K_t_dyn: {K_t_dyn:.3e} N/m")

#  Radiation Dashpot Coefficients 
## Table values
c_z_tab = 0.92 #(table Fig.2(c))
c_y_tab = 0.90 #(table Fig.2(d))
c_rx_tab = 0.08 #(table Fig.2(e))
c_ry_tab = 0.08 #(table Fig.2(f))
c_t_tab = 0.05 #(table Fig.2(g))

## Coefficients
C_z = rho*v_la*A_b*c_z_tab
C_y = rho*v_s*A_b*c_y_tab
C_x = rho*v_s*A_b
C_rx = rho*v_la*I_bx*c_rx_tab
C_ry = rho*v_la*I_by*c_ry_tab
C_t = rho*v_s*I_bz*c_t_tab

## Total Damping (inclusive of materia damping)
C_z_tot = C_z+2*K_z_dyn*beta/omega
C_y_tot = C_y+2*K_y_dyn*beta/omega 
C_x_tot = C_x+2*K_x_dyn*beta/omega  
C_rx_tot = C_rx+2*K_rx_dyn*beta/omega 
C_ry_tot = C_ry+2*K_ry_dyn*beta/omega 
C_t_tot = C_t+2*K_t_dyn*beta/omega 
print("\nTotal Damping Coefficients")
print(f"C_z_tot: {C_z_tot:.3e} N/(m*s)")
print(f"C_y_tot: {C_y_tot:.3e} N/(m*s)")
print(f"C_x_tot: {C_x_tot:.3e} N/(m*s)")
print(f"C_rx_tot: {C_rx_tot:.3e} N/(m*s)")
print(f"C_ry_tot: {C_ry_tot:.3e} N/(m*s)")
print(f"C_t_tot: {C_t_tot:.3e} N/(m*s)")

# Dynamic Stiffness Matrix (inclusive of damping)
K_dyn_vec = [ K_x_dyn, K_y_dyn, K_z_dyn, K_rx_dyn, K_ry_dyn, K_t_dyn ]
C_tot_vec = [ C_x_tot, C_y_tot, C_z_tot, C_rx_tot, C_ry_tot, C_t_tot ]
K_tilde = np.diag([k+1j*omega*c for k,c in zip(K_dyn_vec,C_tot_vec)])
print("\nDynamic Stiffness Matrix (inclusive of damping):")
print(K_tilde)


INPUT RECAP:
L: 2.470e+00 m
B: 2.470e+00 m
A_b: 2.440e+01 m
I_bx: 4.963e+01 m^4
I_by: 4.963e+01 m^4
I_bz: 9.926e+01 m^4
G: 1.920e+08 Pa
v_la: 5.007e+02 m/s
chi: 1.000e+00 -
a_0: 1.502e+00 

Dynamic Stiffness Coefficients:
K_z_dyn: 3.149e+09 N/m
K_y_dyn: 2.505e+09 N/m
K_x_dyn: 2.556e+09 N/m
K_rx_dyn: 1.087e+10 N/m
K_ry_dyn: 9.798e+09 N/m
K_t_dyn: 2.031e+10 N/m

Total Damping Coefficients
C_z_tot: 2.416e+07 N/(m*s)
C_y_tot: 1.495e+07 N/(m*s)
C_x_tot: 1.649e+07 N/(m*s)
C_rx_tot: 9.744e+06 N/(m*s)
C_ry_tot: 9.174e+06 N/(m*s)
C_t_tot: 1.385e+07 N/(m*s)

Dynamic Stiffness Matrix (inclusive of damping):
[[2.55578443e+09+3.10755999e+09j 0.00000000e+00+0.00000000e+00j
  0.00000000e+00+0.00000000e+00j 0.00000000e+00+0.00000000e+00j
  0.00000000e+00+0.00000000e+00j 0.00000000e+00+0.00000000e+00j]
 [0.00000000e+00+0.00000000e+00j 2.50466874e+09+2.81725026e+09j
  0.00000000e+00+0.00000000e+00j 0.00000000e+00+0.00000000e+00j
  0.00000000e+00+0.00000000e+00j 0.00000000e+00+0.00000000e+00j]
 [0.00000